In [7]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

25/03/20 20:49:29 WARN Utils: Your hostname, sherif-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 172.29.81.158 instead (on interface eth0)
25/03/20 20:49:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/20 20:49:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/20 20:49:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/20 20:49:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
spark

In [4]:
sc=spark.sparkContext

In [5]:
from pyspark.sql.functions import *
import pyspark.sql.functions as fn

### read the csv data

In [6]:
df=spark.read.csv('/home/sherif/Downloads/data/sf-fire-calls.csv',header=True,inferSchema=True,
                  samplingRatio=0.001
                 )

In [9]:
df.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: integer (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)


### select medical incidentce

In [13]:
df_medical=df.select('CallNumber','UnitID','IncidentNumber','CallType').where(col('CallType')=='Medical Incident')
df_medical.show()

+----------+------+--------------+----------------+
|CallNumber|UnitID|IncidentNumber|        CallType|
+----------+------+--------------+----------------+
|  20110022|   M17|       2003241|Medical Incident|
|  20110023|   M41|       2003242|Medical Incident|
|  20110197|   E05|       2003343|Medical Incident|
|  20110215|   E06|       2003348|Medical Incident|
|  20110274|   M07|       2003381|Medical Incident|
|  20110304|   E03|       2003399|Medical Incident|
|  20110308|   E14|       2003403|Medical Incident|
|  20110315|   RC2|       2003409|Medical Incident|
|  20110330|   E14|       2003417|Medical Incident|
|  20110330|   M12|       2003417|Medical Incident|
|  20110350|   M41|       2003435|Medical Incident|
|  20110428|   M28|       2003500|Medical Incident|
|  20110467|   T19|       2003529|Medical Incident|
|  20120016|   E43|       2003550|Medical Incident|
|  20120044|   M10|       2003576|Medical Incident|
|  20120045|   E21|       2003577|Medical Incident|
|  20120052|

In [15]:
## another way to write where clause

df_Non_medical=(df.select('CallNumber','UnitID','IncidentNumber','CallType')
                .where('CallType != "Medical Incident"'))
df_Non_medical.show(5)

+----------+------+--------------+--------------+
|CallNumber|UnitID|IncidentNumber|      CallType|
+----------+------+--------------+--------------+
|  20110016|   T13|       2003235|Structure Fire|
|  20110032|   E11|       2003250|  Vehicle Fire|
|  20110043|   B04|       2003259|        Alarms|
|  20110072|   T08|       2003279|Structure Fire|
|  20110125|   E33|       2003301|        Alarms|
+----------+------+--------------+--------------+
only showing top 5 rows



### using describe function to get the table statistics 

In [16]:
df_medical.describe().show()

+-------+--------------------+------------------+--------------------+----------------+
|summary|          CallNumber|            UnitID|      IncidentNumber|        CallType|
+-------+--------------------+------------------+--------------------+----------------+
|  count|              113794|            113794|              113794|          113794|
|   mean|1.0259474883478919E8| 75.11791376319873|1.0137701550986871E7|            NULL|
| stddev| 5.381646466752352E7|12.833009959078586|   5391896.641164449|            NULL|
|    min|             1030145|                27|               30652|Medical Incident|
|    max|           183103638|               UU1|            18130272|Medical Incident|
+-------+--------------------+------------------+--------------------+----------------+



In [18]:
df_medical.describe('CallNumber').show()

+-------+--------------------+
|summary|          CallNumber|
+-------+--------------------+
|  count|              113794|
|   mean|1.0259474883478919E8|
| stddev| 5.381646466752352E7|
|    min|             1030145|
|    max|           183103638|
+-------+--------------------+



### count null and not null values

In [22]:
df.select('CallType')\
.where('CallType is Not Null').count()

175296

In [33]:
## another way
df.select('CallTypeGroup')\
.where(col('CallTypeGroup').isNotNull() ).count()

75779

In [32]:

df.select('CallTypeGroup')\
.count()

175296

In [31]:

df.select('CallTypeGroup')\
.where(col('CallTypeGroup').isNull() ).count()

99517

### distinct values in the column

In [41]:
df.select('CallType')\
.distinct()\
.count()

30

In [42]:
df.select('CallType')\
.distinct()\
.show(30,truncate=False)

+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Elevator / Escalator Rescue                 |
|Marine Fire                                 |
|Aircraft Emergency                          |
|Administrative                              |
|Alarms                                      |
|Odor (Strange / Unknown)                    |
|Citizen Assist / Service Call               |
|HazMat                                      |
|Watercraft in Distress                      |
|Explosion                                   |
|Oil Spill                                   |
|Vehicle Fire                                |
|Suspicious Package                          |
|Extrication / Entrapped (Machinery, Vehicle)|
|Other                                       |
|Outside Fire                                |
|Traffic Collision                           |
|Assist Police                               |
|Gas Leak (Na

In [50]:
df.select('CallType')\
.groupby('CallType')\
.agg(count(col('CallType')).alias('count'))\
.sort('count')\
.show(30,truncate=False)

+--------------------------------------------+------+
|CallType                                    |count |
+--------------------------------------------+------+
|Administrative                              |3     |
|Mutual Aid / Assist Outside Agency          |9     |
|Confined Space / Structure Collapse         |13    |
|Marine Fire                                 |14    |
|Suspicious Package                          |15    |
|Oil Spill                                   |21    |
|Watercraft in Distress                      |28    |
|Extrication / Entrapped (Machinery, Vehicle)|28    |
|High Angle Rescue                           |32    |
|Assist Police                               |35    |
|Aircraft Emergency                          |36    |
|Train / Rail Incident                       |57    |
|Explosion                                   |89    |
|Industrial Accidents                        |94    |
|HazMat                                      |124   |
|Fuel Spill                 

### Rename column

In [61]:
df2=df.withColumnRenamed('Delay','Response_delay')
df2.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: integer (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)


In [99]:
df_response=df2.select('City','Response_delay')\
.where('Response_delay > 3')
df_response.show()

+----+--------------+
|City|Response_delay|
+----+--------------+
|  SF|           4.7|
|  SF|     3.4833333|
|  SF|          4.95|
|  SF|          5.35|
|  SF|     4.4666667|
|  SF|     3.8833334|
|  SF|           3.3|
|  SF|           3.1|
|  SF|     3.3333333|
|  SF|     3.9666667|
|  SF|          3.25|
|  SF|     4.2833333|
|  SF|          6.25|
|  SF|     3.6166666|
|  SF|          4.65|
|  SF|     4.1833334|
|  SF|          3.55|
|  SF|     4.5333333|
|  SF|     3.1666667|
|  SF|      4.766667|
+----+--------------+
only showing top 20 rows



### convert to pandas dataframe

In [100]:
df_pandas=df_response.toPandas()

In [103]:
import pandas as pd
df_pandas.head()

,City,Response_delay
0,SF,4.700000
1,SF,3.483333
2,SF,4.950000
3,SF,5.350000
4,SF,4.466667


In [108]:
df_pandas['City'].unique()

array(['SF', 'HP', 'TI', 'SFO', 'PR', 'BN', 'YB', 'FM', 'DC', 'OAK',
       'SAN FRANCISCO', None, 'PRESIDIO', 'TREASURE ISLAND', 'FORT MASON',
       'San Francisco', 'Presidio', 'Treasure Island', 'Hunters Point',
       'Treasure Isla', 'Fort Mason', 'Yerba Buena', 'Brisbane',
       'Daly City'], dtype=object)

### average dealy in calls for every city

In [76]:
df.select('city').distinct().count()

24

In [77]:
df2.select('city','Response_delay')\
.where('Response_delay > 5')\
.groupby('city')\
.agg(avg('Response_delay').alias('avg_delay'))\
.sort('avg_delay')\
.show(24)

+---------------+------------------+
|           city|         avg_delay|
+---------------+------------------+
|            OAK|         5.5916668|
|       Brisbane|        5.65833325|
|  Hunters Point|      5.6749999125|
|TREASURE ISLAND|        6.48749995|
|             HP| 6.986111233333332|
|Treasure Island|        7.39333354|
|             FM| 7.489583512499999|
|    Yerba Buena| 7.805555523809524|
|     Fort Mason| 8.315000049999998|
|             BN|       8.495833075|
|       Presidio| 8.734074042222222|
|             SF| 9.051151857876587|
|             TI| 9.513827127407408|
|  Treasure Isla| 9.573290621794873|
|            SFO|      10.004166625|
|  San Francisco|10.677915514264303|
|             PR|    10.72630201875|
|  SAN FRANCISCO|11.126889518313961|
|             DC|13.507143214285716|
|      Daly City|14.952777500000002|
|             YB| 19.53555526666667|
|       PRESIDIO|         33.133335|
|           NULL| 53.38120545106383|
+---------------+------------------+



### change data type

In [79]:
df.select('callDate').show(10)

+----------+
|  callDate|
+----------+
|01/11/2002|
|01/11/2002|
|01/11/2002|
|01/11/2002|
|01/11/2002|
|01/11/2002|
|01/11/2002|
|01/11/2002|
|01/11/2002|
|01/11/2002|
+----------+
only showing top 10 rows



In [90]:
df_date_formate=df.withColumn('Date\time',to_timestamp(col('callDate'),'mm/dd/yyyy'))\
.drop('callDate')
df_date_formate.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: integer (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- SupervisorDistrict: string (nullabl

In [93]:
year=df_date_formate.select(fn.year('Date\time').alias('year'))
year.show()

+----+
|year|
+----+
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
+----+
only showing top 20 rows



In [98]:
year.select('year').distinct().sort('year').show()

+----+
|year|
+----+
|2000|
|2001|
|2002|
|2003|
|2004|
|2005|
|2006|
|2007|
|2008|
|2009|
|2010|
|2011|
|2012|
|2013|
|2014|
|2015|
|2016|
|2017|
|2018|
+----+



# write sql quieres in spark 

In [117]:
from pyspark.sql.types import *
schem=StructType([StructField('date', StringType(), True), 
            StructField('delay', IntegerType(), True), 
            StructField('distance', IntegerType(), True), 
            StructField('origin', StringType(), True),
            StructField('destination', StringType(), True)])



In [118]:
data=spark.read.csv('/home/sherif/Downloads/data/departuredelays.csv',header=True,schema=schem)
data.show(6)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 6 rows



In [120]:
data.printSchema()

root
 |-- date: string (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [128]:
data.createOrReplaceTempView('delays_df')

In [129]:
df_dalay=data.select('*').where('delay > 5')
df_dalay.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
|01061725|   69|     602|   ABE|        ATL|
|01081230|   33|     369|   ABE|        DTW|
|01081219|   54|     569|   ABE|        ORD|
|01091215|   43|     602|   ABE|        ATL|
|01090600|  151|     369|   ABE|        DTW|
|01090625|    8|     602|   ABE|        ATL|
|01091219|   83|     569|   ABE|        ORD|
|01101725|    7|     602|   ABE|        ATL|
|01100625|   52|     602|   ABE|        ATL|
|01111215|  127|     602|   ABE|        ATL|
|01131215|   14|     602|   ABE|        ATL|
|01130625|   29|     602|   ABE|        ATL|
|01161219|   68|     569|   ABE|        ORD|
|01210600|   89|     369|   ABE|        DTW|
|01211230|

#### now using sql  quiery

In [131]:
df_delay_sql= spark.sql('SELECT * FROM delays_df WHERE delay > 5')
df_delay_sql.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
|01061725|   69|     602|   ABE|        ATL|
|01081230|   33|     369|   ABE|        DTW|
|01081219|   54|     569|   ABE|        ORD|
|01091215|   43|     602|   ABE|        ATL|
|01090600|  151|     369|   ABE|        DTW|
|01090625|    8|     602|   ABE|        ATL|
|01091219|   83|     569|   ABE|        ORD|
|01101725|    7|     602|   ABE|        ATL|
|01100625|   52|     602|   ABE|        ATL|
|01111215|  127|     602|   ABE|        ATL|
|01131215|   14|     602|   ABE|        ATL|
|01130625|   29|     602|   ABE|        ATL|
|01161219|   68|     569|   ABE|        ORD|
|01210600|   89|     369|   ABE|        DTW|
|01211230|

In [138]:
spark.sql('''SELECT * FROM delays_df 
WHERE delay > 30 AND origin = "SFO" AND destination="ORD" 
ORDER BY delay DESC
''').show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|02190925| 1638|    1604|   SFO|        ORD|
|01031755|  396|    1604|   SFO|        ORD|
|01022330|  326|    1604|   SFO|        ORD|
|01051205|  320|    1604|   SFO|        ORD|
|01190925|  297|    1604|   SFO|        ORD|
|02171115|  296|    1604|   SFO|        ORD|
|01071040|  279|    1604|   SFO|        ORD|
|01051550|  274|    1604|   SFO|        ORD|
|03120730|  266|    1604|   SFO|        ORD|
|01261104|  258|    1604|   SFO|        ORD|
|01161210|  225|    1604|   SFO|        ORD|
|02091800|  223|    1604|   SFO|        ORD|
|01221040|  215|    1604|   SFO|        ORD|
|03121155|  203|    1604|   SFO|        ORD|
|02111256|  197|    1604|   SFO|        ORD|
|03311405|  196|    1604|   SFO|        ORD|
|01031920|  193|    1604|   SFO|        ORD|
|01021410|  190|    1604|   SFO|        ORD|
|03171215|  189|    1604|   SFO|        ORD|
|01101410|

25/03/21 13:08:30 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3033106 ms exceeds timeout 120000 ms
25/03/21 13:08:31 WARN SparkContext: Killing executors is not supported by current scheduler.
25/03/21 13:08:31 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 